## Perform installs and imports


*   Specify json schema for text completion outputs.


In [ ]:
!pip install openai pyairtable

In [ ]:
import openai, json
from openai import OpenAI
from pydantic import BaseModel

from google.colab import userdata
openai.api_key = userdata.get("OPENAI_API_KEY")

class output(BaseModel):
  translationtext: str
  extracommentary: str

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import display, update_display

## Take text to translate

In [ ]:
confucius = """不亦樂乎
"""

In [ ]:
confucius2 = """
子曰：「學而時習之，不亦說乎？有朋自遠方來，不亦樂乎？人不知而不慍，不亦君子乎？
"""

In [ ]:
hyungdo = """엄마 걱정
기형도
열무 삼십 단을 이고
시장에 간 우리 엄마
안 오시네, 해는 시든 지 오래
나는 찬밥처럼 방에 담겨
아무리 천천히 숙제를 해도
엄마 안 오시네, 배춧잎 같은 발소리 타박타박
안 들리네, 어둡고 무서워
금간 창 틈으로 고요히 빗소리
빈 방에 혼자 엎드려 훌쩍거리던
아주 먼 옛날
지금도 내 눈시울을 뜨겁게 하는
그 시절, 내 유년의 윗목 """

In [ ]:
leopardi = """ O graziosa luna, io mi rammento
Che, or volge l’anno, sovra questo colle
Io venia pien d’angoscia a rimirarti:
E tu pendevi allor su quella selva
Siccome or fai, che tutta la rischiari.
Ma nebuloso e tremulo dal pianto
Che mi sorgea sul ciglio, alle mie luci
Il tuo volto apparia, che travagliosa
Era mia vita: ed è, nè cangia stile,
O mia diletta luna. E pur mi giova
La ricordanza, e il noverar l’etate
Del mio dolore. Oh come grato occorre
Nel tempo giovanil, quando ancor lungo
La speme e breve ha la memoria il corso,
Il rimembrar delle passate cose,
Ancor che triste, e che l’affanno duri! """

In [ ]:
wittgenstein = """ 1
Die Welt ist alles, was der Fall ist.
Die Welt ist die Gesamtheit der Tatsachen, nicht der Dinge.
Die Welt ist durch die Tatsachen bestimmt und dadurch, dass es alle Tatsachen sind.
Denn, die Gesamtheit der Tatsachen bestimmt, was der Fall ist und auch, was alles nicht der Fall ist.
Die Tatsachen im logischen Raum sind die Welt."""

## Define functions to carry out "ping pong"


*   First function: text completion that carries out a "free" translation; no specified elements of the text to take into account.
*   Second and third functios: recto and verso; carry out translation back and forth between the languages.



In [ ]:
client = OpenAI(api_key=openai.api_key)

In [ ]:
text0 = "manger" # set to one of the passages above, or import your own

In [ ]:
def translatebasic(outputlang, inputtext):
    response = client.responses.parse(
        model="o3",
        input=[
            {"role": "system", "content": "You translate strings verbatim. "},
            {
                "role": "user",
                "content": f"Translate the following text to {outputlang}. Text: {inputtext}. Just output the text, absolutely nothing before or after. Do not add punctuation to your output. ",
            },
        ],
        text_format=output,
    )

    final = response.output_parsed.translationtext
    return(final)

lang 0 is english, lang 1 is french

In [ ]:
def tolang0(text):
    res1 = translatebasic("english", text0)
    return res1

def tolang1(text):
    res2 = translatebasic("french", text0)
    return res2

## Carry out "ping pong":

*  Define number of trials for ping pong (one trial is one back and one forth).
*   Carry out use a for loop to test for convergence.

In [ ]:
trials = 50

In [ ]:
for i in range(trials):
  text1 = tolang0(text0)
  print(text1)
  text0 = tolang1(text1)
  print(text0)

In [ ]:
def char_mismatch_count(a: str, b: str) -> int:
    la, lb = len(a), len(b)
    n = max(la, lb)
    mismatches = 0
    for i in range(n):
        ca = a[i] if i < la else '\0'
        cb = b[i] if i < lb else '\0'
        if ca != cb:
            mismatches += 1
    return mismatches

In [ ]:
text0 = "je mange une pomme"
print(text0)

In [ ]:
trials = 50
seed_text_lang2 = text0

english_diffs = []
french_diffs  = []

prev_en = None
prev_fr = seed_text_lang2

fig, ax = plt.subplots(figsize=(8, 5))
(en_line,) = ax.plot([], [], label="Language 0 differences")
(fr_line,) = ax.plot([], [], label="Language 1 differences")
ax.set_xlabel("# iterations")
ax.set_ylabel("# differences")
ax.set_title("Ping-pong differences (live)")
ax.grid(True, alpha=0.3)
ax.legend()
display(fig, display_id="live_plot")

for i in range(1, trials + 1):

    en_i = tolang1(prev_fr)
    print(en_i)

    if prev_en is None:
        d_en = 0
    else:
        d_en = char_mismatch_count(en_i, prev_en)
    english_diffs.append(d_en)
    prev_en = en_i

    fr_i = tolang0(en_i)
    print(fr_i)

    d_fr = char_mismatch_count(fr_i, prev_fr)
    french_diffs.append(d_fr)
    prev_fr = fr_i

    print(i, d_en, d_fr)

    iters = list(range(1, i + 1))
    en_line.set_data(iters, english_diffs)
    fr_line.set_data(iters, french_diffs)

    ax.set_xlim(1, trials)
    y_max = max(1, max(english_diffs + french_diffs))
    ax.set_ylim(0, y_max * 1.1)

    fig.canvas.draw()
    update_display(fig, display_id="live_plot")

print("EN:", english_diffs)
print("FR:", french_diffs)